In [ ]:
import os
import zipfile
import random
import shutil
import tensorflow as tf
#import tensorflowjs as tfjs
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from shutil import copyfile
import matplotlib.pyplot as plt

In [ ]:
!wget --no-check-certificate \
    "https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip" \
    -O "/tmp/cats-and-dogs.zip"

local_zip = 'tmp/cats-and-dogs.zip'
zip_ref   = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/tmp')
zip_ref.close()

In [2]:
TRAINING_DIR = "tmp/cats_and_dogs_filtered/train/"
VALIDATION_DIR = "tmp/cats_and_dogs_filtered/validation/"

In [3]:

def train_val_generators(TRAINING_DIR, VALIDATION_DIR):

  train_datagen = ImageDataGenerator(rescale=1./255,
                                        #rotation_range=40,
                                        #width_shift_range=0.2,
                                        #height_shift_range=0.2,
                                        #shear_range=0.2,
                                        #zoom_range=0.2,
                                        #horizontal_flip=True,
                                        #fill_mode='nearest'
                                        )


  train_generator = train_datagen.flow_from_directory(directory=TRAINING_DIR,
                                                      batch_size=20,
                                                      class_mode='binary',
                                                      target_size=(150, 150))

  validation_datagen = ImageDataGenerator(rescale=1./255)


  validation_generator = validation_datagen.flow_from_directory(directory=VALIDATION_DIR,
                                                                #batch_size=20,
                                                                class_mode='binary',
                                                                target_size=(150, 150))
  return train_generator, validation_generator

In [ ]:
train_generator, validation_generator = train_val_generators(TRAINING_DIR, VALIDATION_DIR)

In [5]:
def create_model():

  model = tf.keras.models.Sequential([
      tf.keras.layers.Conv2D(64, (3,3), activation='relu', input_shape=(150,150,3)),
      tf.keras.layers.MaxPooling2D(2,2),
      tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
      tf.keras.layers.MaxPooling2D(2,2),
      tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
      tf.keras.layers.MaxPooling2D(2,2),
      tf.keras.layers.Flatten(),
      tf.keras.layers.Dense(256, activation= 'relu'),
      tf.keras.layers.Dense(64, activation= 'relu'),
      tf.keras.layers.Dense(64, activation= 'relu'),
      tf.keras.layers.Dense(2, activation='softmax'),
  ])


  model.compile(optimizer='adam',
                loss='sparse_categorical_crossentropy',
                metrics=['accuracy'])


  return model

In [ ]:
model = create_model()

class myCallback(tf.keras.callbacks.Callback): 
    def on_epoch_end(self, epoch, logs={}):
        if logs.get('accuracy') is not None and logs.get('accuracy') > 0.95:
            print("\nDone reached 95.0% accuracy so we done here ig....")
            self.model.stop_training = True

callbacks = myCallback()
history = model.fit(train_generator,
                    epochs=50,
                    verbose=1,
                    callbacks=[callbacks],
                    validation_data=validation_generator)

In [ ]:

acc=history.history['accuracy']
val_acc=history.history['val_accuracy']
loss=history.history['loss']
val_loss=history.history['val_loss']

epochs=range(len(acc)) 

plt.plot(epochs, acc, 'r', "Training Accuracy")
plt.plot(epochs, val_acc, 'b', "Validation Accuracy")
plt.title('Training and validation accuracy')
plt.show()
print("")

plt.plot(epochs, loss, 'r', "Training Loss")
plt.plot(epochs, val_loss, 'b', "Validation Loss")
plt.show()

In [ ]:
model.save('catsanddogs_model.h5')
model.save('catsanddogs_model.keras', save_format='keras')
model.save("catsanddogs_SavedModel", save_format='tf')
#tfjs.converters.save_keras_model(model, "catsanddogs_TensorFlowJS")

converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()
with open("catsanddogs.tflite", "wb") as f:
    f.write(tflite_model)